In [1]:
from keras.models import Model, load_model
from keras.engine.network import Network
from keras.layers import Input
import cv2
import numpy as np

Using TensorFlow backend.


In [2]:
fname = 'EMVPpPE.mp4'
path = 'data/' + fname

In [3]:
cap = cv2.VideoCapture(path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 動画の画面横幅
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 動画の画面縦幅
fcnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # 総フレーム数
fps = cap.get(cv2.CAP_PROP_FPS)

ret, frame = cap.read()
low_frame = cv2.resize(frame , (int(w/2), int(h/2)))

In [4]:
pre_model = load_model('model/20200511_generator.h5')
# pre_model = srgan.generator
pre_net = Network(inputs=pre_model.input, outputs=pre_model.get_layer(pre_model.layers[-1].name).output, name='learned')

input_layer = Input(shape = (540, 960, 3))
sr_out = pre_net(input_layer)
sr_generator = Model(input_layer, sr_out)

c:\users\ryoji\appdata\local\programs\python\python37\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
sr_generator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 540, 960, 3)       0         
_________________________________________________________________
learned (Network)            multiple                  240451    
Total params: 240,451
Trainable params: 239,683
Non-trainable params: 768
_________________________________________________________________


In [ ]:
img = low_frame
img_gen = sr_generator.predict(img[np.newaxis, :, :, :])

img_int = ((img_gen * 127.5) + 127.5).astype('u1').reshape((1080, 1920, 3))
cv2.imwrite('data/20200511_out_premerged.jpg', img_int)